# Sentiment Analysis with RNN

Reference:
- https://towardsdatascience.com/a-beginners-guide-on-sentiment-analysis-with-rnn-9e100627c02e

In [1]:
from keras.datasets import imdb

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/grace/workspace/keras/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
vocab_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

In [3]:
X_train.shape

(25000,)

In [4]:
X_test.shape

(25000,)

In [5]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [6]:
[id2word.get(i, '') for i in X_train[6]]

['the',
 'and',
 'full',
 'involving',
 'to',
 'impressive',
 'boring',
 'this',
 'as',
 'and',
 'and',
 'br',
 'villain',
 'and',
 'and',
 'need',
 'has',
 'of',
 'costumes',
 'b',
 'message',
 'to',
 'may',
 'of',
 'props',
 'this',
 'and',
 'and',
 'concept',
 'issue',
 'and',
 'to',
 "god's",
 'he',
 'is',
 'and',
 'unfolds',
 'movie',
 'women',
 'like',
 "isn't",
 'surely',
 "i'm",
 'and',
 'to',
 'toward',
 'in',
 "here's",
 'for',
 'from',
 'did',
 'having',
 'because',
 'very',
 'quality',
 'it',
 'is',
 'and',
 'and',
 'really',
 'book',
 'is',
 'both',
 'too',
 'worked',
 'carl',
 'of',
 'and',
 'br',
 'of',
 'reviewer',
 'closer',
 'figure',
 'really',
 'there',
 'will',
 'and',
 'things',
 'is',
 'far',
 'this',
 'make',
 'mistakes',
 'and',
 'was',
 "couldn't",
 'of',
 'few',
 'br',
 'of',
 'you',
 'to',
 "don't",
 'female',
 'than',
 'place',
 'she',
 'to',
 'was',
 'between',
 'that',
 'nothing',
 'and',
 'movies',
 'get',
 'are',
 'and',
 'br',
 'yes',
 'female',
 'just

In [7]:
#max review length
len(max((X_train + X_test), key=len))

2569

In [8]:
#min review length
len(min((X_train + X_test), key=len))

69

## pad sequences

In [9]:
from keras.preprocessing import sequence

max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

## Model : RNN

In [10]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [11]:
embedding_size = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
batch_size = 64
num_epoches = 30
validation_split = 0.2

In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

early_stopping_callback = EarlyStopping(monitor='val_loss', \
                                       patience=100)

In [16]:
# X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
# X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
# history = model.fit(X_train2, y_train2,\
#                     validation_split=(X_valid, y_valid),\
#                    batch_size=batch_size, epochs=num_epoches)

In [17]:
history = model.fit(X_train, y_train,\
                   batch_size=batch_size, epochs=num_epoches,\
                   verbose=1, validation_split=validation_split,\
                   callbacks=[early_stopping_callback])

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 231s - loss: 0.5407 - acc: 0.7393 - val_loss: 0.3872 - val_acc: 0.8362
Epoch 2/30
20000/20000 [==============================] - 224s - loss: 0.3415 - acc: 0.8593 - val_loss: 0.3626 - val_acc: 0.8476
Epoch 3/30
20000/20000 [==============================] - 225s - loss: 0.2844 - acc: 0.8868 - val_loss: 0.3422 - val_acc: 0.8578
Epoch 4/30
20000/20000 [==============================] - 233s - loss: 0.2481 - acc: 0.9073 - val_loss: 0.3395 - val_acc: 0.8590
Epoch 5/30
20000/20000 [==============================] - 228s - loss: 0.2143 - acc: 0.9190 - val_loss: 0.3405 - val_acc: 0.8626
Epoch 6/30
20000/20000 [==============================] - 30381s - loss: 0.2005 - acc: 0.9267 - val_loss: 0.3651 - val_acc: 0.8476
Epoch 7/30
20000/20000 [==============================] - 223s - loss: 0.2064 - acc: 0.9251 - val_loss: 0.3598 - val_acc: 0.8540
Epoch 8/30
20000/20000 [======================

In [18]:
score = model.evaluate(X_test, y_test, verbose=1)
print('\n')
print('test score : ', score[0])
print('test accuracy : ', score[1])

25000/25000 [==============================] - 133s   


test score :  1.0346745039898158
test accuracy :  0.82944
